In [1]:
import sys
import os
from pathlib import Path
path = Path(os.path.abspath(''))
sys.path.append(str(path.parents[0]))

from pyvesto import pipeline, image

In [2]:
input_path = Path('../data/images')
output_path = Path('../data/results')

In [3]:
dp = pipeline.build_default_pipeline(input_path, output_path)

In [ ]:
dp.run()